In [ ]:
import sqlite3
import random

class Musteri:
    def __init__(self, musteri_numarasi, ad, soyad):
        self.musteri_numarasi = musteri_numarasi
        self.ad = ad
        self.soyad = soyad

    def bilgileri_goster(self):
        print("\nMüşteri Bilgileri:")
        print(f"""
            Müşteri Numarası: {self.musteri_numarasi}
            Ad: {self.ad}
            Soyad: {self.soyad}
        """)

class Hesap:
    def __init__(self, hesap_numarasi, musteri, bakiye=0):
        self.hesap_numarasi = hesap_numarasi
        self.musteri = musteri
        self.bakiye = bakiye
        self.acik_hesap = True

    def para_yatirmak(self, miktar):
        if self.acik_hesap and miktar > 0:
            self.bakiye += miktar
            self.guncelle_hesap()
            print(f"""
                Yatırdığınız Miktar: {miktar}
                Şuanki Hesap Bakiyeniz: {self.bakiye}
            """)
        elif not self.acik_hesap:
            print("Hesap Kapalıdır.")
        else:
            print("Girdiğiniz miktar yanlıştır. Lütfen geçerli bir miktar giriniz.")

    def para_cekmek(self, miktar):
        if self.acik_hesap and miktar > 0 and miktar <= self.bakiye:
            self.bakiye -= miktar
            self.guncelle_hesap()
            print(f"""
                Çekmek İstediğiniz Miktar: {miktar}
                Şuanki Hesap Bakiyeniz: {self.bakiye}
            """)
        elif not self.acik_hesap:
            print("Hesap Kapalıdır.")
        elif miktar > self.bakiye:
            print("Girmiş olduğunuz miktar hesap bakiyenizden fazladır. Lütfen geçerli bir miktar giriniz.")
        else:
            print("Girdiğiniz miktar yanlıştır. Lütfen geçerli bir miktar giriniz.")

    def bakiye_sorgulama(self):
        if self.acik_hesap:
            print(f"Şuanki hesap bakiyeniz: {self.bakiye}")
        else:
            print("Hesap Kapalıdır.")

    def hesap_kapatmak(self):
        if self.acik_hesap:
            self.acik_hesap = False
            self.guncelle_hesap()
            print("Hesap başarılı bir şekilde kapatıldı.")
        else:
            print("Hesap Zaten Kapalıdır.")

    def hesap_aktif_etmek(self):
        if not self.acik_hesap:
            self.acik_hesap = True
            self.guncelle_hesap()
            print("Hesap başarılı bir şekilde aktif edildi.")
        else:
            print("Hesap Zaten Aktiftir.")

    def bilgileri_goster(self):
        print(f"\nHesap Bilgileri: ")
        print(f"""
            Hesap Numarası: {self.hesap_numarasi}
            Hesap Bakiyesi: {self.bakiye}
        """)
        self.musteri.bilgileri_goster()

    def guncelle_hesap(self):
        connection = sqlite3.connect("banka.db")
        cursor = connection.cursor()

        cursor.execute("UPDATE Hesaplar SET bakiye = ?, acik_hesap = ? WHERE hesap_numarasi = ?",
                       (self.bakiye, 1 if self.acik_hesap else 0, self.hesap_numarasi))

        connection.commit()
        connection.close()

class Banka:
    def __init__(self):
        self.baglanti_olustur()

    def baglanti_olustur(self):
        self.connection = sqlite3.connect("banka.db")
        self.cursor = self.connection.cursor()

        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Musteriler (
                        musteri_numarasi VARCHAR(11) PRIMARY KEY,
                        ad VARCHAR(100),
                        soyad VARCHAR(100)
                        )''')

        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Hesaplar (
                        hesap_numarasi INTEGER PRIMARY KEY,
                        musteri_numarasi VARCHAR(11),
                        bakiye REAL,
                        acik_hesap BIT,
                        FOREIGN KEY (musteri_numarasi) REFERENCES Musteriler(musteri_numarasi)
                        )''')

        self.connection.commit()

    def baglanti_kapat(self):
        self.connection.close()

    def hesap_olusturmak(self, musteri, ilk_bakiye=0):
        hesap_numarasi = random.randint(1000,99999999)
        hesap = Hesap(hesap_numarasi, musteri, ilk_bakiye)
        self.kaydet_musteri(musteri)
        self.kaydet_hesap(hesap)
        print(f"{hesap_numarasi} numaralı hesap başarılı bir şekilde oluşturuldu.")
        return hesap

    def kaydet_musteri(self, musteri):
        self.cursor.execute("INSERT INTO Musteriler VALUES (?, ?, ?)", (musteri.musteri_numarasi, musteri.ad, musteri.soyad))
        self.connection.commit()

    def kaydet_hesap(self, hesap):
        self.cursor.execute("INSERT INTO Hesaplar VALUES (?, ?, ?, ?)",
                            (hesap.hesap_numarasi, hesap.musteri.musteri_numarasi, hesap.bakiye, 1 if hesap.acik_hesap else 0))
        self.connection.commit()

    def tum_hesaplar(self):
        print("\nTüm Hesaplar: ")
        self.cursor.execute("SELECT * FROM Hesaplar")
        hesap_rows = self.cursor.fetchall()

        for row in hesap_rows:
            hesap_numarasi, musteri_numarasi, bakiye, acik_hesap = row

            musteri = self.musteri_getir(musteri_numarasi)
            hesap = Hesap(hesap_numarasi, musteri, bakiye)
            hesap.acik_hesap = True if acik_hesap == 1 else False

            hesap.bilgileri_goster()

    def tum_musteriler(self):
        print("\nTüm Müşteriler: ")
        self.cursor.execute("SELECT * FROM Musteriler")
        musteri_rows = self.cursor.fetchall()

        for row in musteri_rows:
            musteri_numarasi, ad, soyad = row
            musteri = Musteri(musteri_numarasi, ad, soyad)
            musteri.bilgileri_goster()

    def hesap_numarasi_kontrolu(self):
        hesap_numarasi = int(input("Hesap Numarasını Giriniz: "))
        self.cursor.execute("SELECT * FROM Hesaplar WHERE hesap_numarasi = ?", (hesap_numarasi,))
        hesap_row = self.cursor.fetchone()

        if hesap_row:
            musteri = self.musteri_getir(hesap_row[1])
            hesap = Hesap(hesap_row[0], musteri, hesap_row[2])
            hesap.acik_hesap = True if hesap_row[3] == 1 else False
            return hesap
        else:
            print("Hesap Bulunamadı.")
            return None

    def musteri_getir(self, musteri_numarasi):
        self.cursor.execute("SELECT * FROM Musteriler WHERE musteri_numarasi = ?", (musteri_numarasi,))
        musteri_row = self.cursor.fetchone()

        if musteri_row:
            musteri = Musteri(musteri_row[0], musteri_row[1], musteri_row[2])
            return musteri
        else:
            return None

banka = Banka()

try:
    while True:
        print(20 * "*", "Banka Hesap Yönetim Sistemi Menüsü", 20 * "*")
        print("""
            1. Hesap Oluşturmak
            2. Tüm Hesaplar Listesi
            3. Tüm Müşteriler Listesi
            4. Para Yatırmak
            5. Para Çekmek
            6. Bakiye Sorgulama
            7. Hesap Kapatmak 
            8. Hesap Aktif Etmek 
            9. Çıkış (q - Q)
        """)

        islem = input("Lütfen yapacağınız işlemin numarasını giriniz: ")

        if islem == "1":
            musteri_numarasi = input("Müşteri Numarası: ")
            ad = input("Ad: ")
            soyad = input("Soyad: ")
            musteri = Musteri(musteri_numarasi, ad, soyad)
            banka.hesap_olusturmak(musteri)

        elif islem == "2":
            banka.tum_hesaplar()

        elif islem == "3":
            banka.tum_musteriler()

        elif islem == "4":
            hesap = banka.hesap_numarasi_kontrolu()
            if hesap:
                miktar = float(input("Yatıracağınız Miktarı Giriniz: "))
                hesap.para_yatirmak(miktar)

        elif islem == "5":
            hesap = banka.hesap_numarasi_kontrolu()
            if hesap:
                miktar = float(input("Çekmek İstediğiniz Miktarı Giriniz: "))
                hesap.para_cekmek(miktar)

        elif islem == "6":
            hesap = banka.hesap_numarasi_kontrolu()
            if hesap:
                hesap.bakiye_sorgulama()

        elif islem == "7":
            hesap = banka.hesap_numarasi_kontrolu()
            if hesap:
                hesap.hesap_kapatmak()

        elif islem == "8":
            hesap = banka.hesap_numarasi_kontrolu()
            if hesap:
                hesap.hesap_aktif_etmek()

        elif islem == "9" or islem.lower() == "q":
            print("Uygulamadan Çıkış Yapıldı.")
            break

        else:
            print("Yanlış bir işlem seçimi yaptınız.")
finally:
    banka.baglanti_kapat()